In [10]:
import pandas as pd
import requests
import json

In [11]:
loc = 'C:\\Users\\Darrick\\Documents\\Jupyter\\senior-project\\stats\\'
season = '2021-22'

In [12]:
headers = {
    'Accept': 'application/json, text/plain, */*',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36',
    'x-nba-stats-origin': 'stats',
    'Sec-Fetch-Site': 'same-site',
    'Sec-Fetch-Mode': 'cors',
    'Referer': 'https://www.nba.com/'
}

# Retrieve both defensive dashboard and defensive impact
r_dd = requests.get('https://stats.nba.com/stats/leaguedashptdefend?College=&Conference=&Country=&DateFrom=&DateTo=&DefenseCategory=Overall&Division=&DraftPick=&DraftYear=&GameSegment=&Height=&LastNGames=0&LeagueID=00&Location=&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PerMode=PerGame&Period=0&PlayerExperience=&PlayerPosition=&Season=' + season + '&SeasonSegment=&SeasonType=Regular+Season&StarterBench=&TeamID=0&VsConference=&VsDivision=&Weight=', headers=headers, timeout=10)
r_di = requests.get('https://stats.nba.com/stats/leaguedashptstats?College=&Conference=&Country=&DateFrom=&DateTo=&Division=&DraftPick=&DraftYear=&GameScope=&Height=&LastNGames=0&LeagueID=00&Location=&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PerMode=PerGame&PlayerExperience=&PlayerOrTeam=Player&PlayerPosition=&PtMeasureType=Defense&Season=' + season + '&SeasonSegment=&SeasonType=Regular+Season&StarterBench=&TeamID=0&VsConference=&VsDivision=&Weight=', headers=headers, timeout=10)

In [13]:
r_dd_dict = json.loads(r_dd.content)
r_di_dict = json.loads(r_di.content)

dd_df = pd.DataFrame(data=r_dd_dict['resultSets'][0]['rowSet'], columns=r_dd_dict['resultSets'][0]['headers'])
di_df = pd.DataFrame(data=r_di_dict['resultSets'][0]['rowSet'], columns=r_di_dict['resultSets'][0]['headers'])

dd_df.sort_values(by='PLAYER_NAME', inplace=True)
di_df.sort_values(by='PLAYER_NAME', inplace=True)

In [18]:
# Remove players not in both dataframes
diff = list(set(di_df['PLAYER_ID'].tolist()) - set(dd_df['CLOSE_DEF_PERSON_ID'].tolist()))

di_df = di_df[~di_df['PLAYER_ID'].isin(diff)]

# Combine dataframes
def_data = dd_df.join(di_df[di_df.columns[5:11]])

# Drop unnecessary columns
def_data.drop(columns=['PLAYER_LAST_TEAM_ID', 'PLAYER_LAST_TEAM_ABBREVIATION'], inplace=True)
def_data.rename(columns={'CLOSE_DEF_PERSON_ID' : 'PLAYER_ID'}, inplace=True)

def_data.head()

,PLAYER_ID,PLAYER_NAME,PLAYER_POSITION,AGE,GP,G,FREQ,D_FGM,D_FGA,D_FG_PCT,NORMAL_FG_PCT,PCT_PLUSMINUS,W,L,MIN,STL,BLK,DREB
129,203932,Aaron Gordon,F,26.0,44,44,1.0,4.91,11.27,0.435,0.459,-0.024,14.0,15.0,13.0,0.3,0.9,4.0
557,1630565,Aaron Henry,F,22.0,5,5,1.0,0.40,1.60,0.250,0.405,-0.155,11.0,27.0,15.6,0.7,0.1,1.1
359,1628988,Aaron Holiday,G,25.0,34,34,1.0,3.00,6.29,0.477,0.434,0.043,4.0,3.0,34.6,1.0,0.3,4.4
464,1630174,Aaron Nesmith,G-F,22.0,33,33,1.0,2.03,4.09,0.496,0.428,0.068,3.0,10.0,2.9,0.1,0.1,0.5
213,1630598,Aaron Wiggins,G,23.0,29,29,1.0,4.59,9.21,0.498,0.456,0.042,2.0,6.0,21.4,0.6,0.1,2.5


In [19]:
def_data.to_csv(loc + season + '_def_stats.csv', index=False)